# DAY 1

In [115]:
# Part1

import polars as pl

txt = open('files/advent1_1.txt', 'r').readlines()
l1 = [int(r.split(' ')[0].strip()) for r in txt]
l2 = [int(r.split(' ')[-1].strip()) for r in txt]
df1, df2 = pl.DataFrame({'l1': l1}, schema={'l1': pl.Int64}).sort('l1', descending=False).with_row_index(), pl.DataFrame({'l2': l2}, schema={'l2': pl.Int64}).sort('l2', descending=False).with_row_index()
df1.join(df2, on='index').with_columns(abs(pl.col('l1') - pl.col('l2')).alias('dif'))['dif'].sum()

1646452

In [116]:
# Part2
(df1.join(df2.group_by('l2').agg(pl.len()), left_on='l1', right_on='l2', how='left')
 .fill_null(0)
 .with_columns((pl.col('l1') * pl.col('len')).alias('sim'))
 ['sim'].sum())

23609874

# DAY 2

In [117]:
txt = [[int(d) for d in l.split(' ')] for l in open('files/advent2.txt', 'r').read().split('\n')]

In [129]:
# Part 1
def is_false(row):
    incr = [row[i] - row[i + 1] for i in range(0, len(row) - 1)]
    return 0 if (set(incr) <= {1, 2, 3} or set(incr) <= {-1, -2, -3}) else 1

len(txt) - sum(is_false(row) for row in txt)

660

In [130]:
# Part 2
len(txt) - sum(all(is_false(row[:i] + row[i + 1:]) for i in range(len(row))) for row in txt)

689

# DAY 3

In [3]:
# Part 1
import re

sum(list(map(lambda l: int(l[0]) * int(l[1]), re.findall(r'mul\((\d{1,3}),(\d{1,3})\)', open('files/advent3.txt', 'r').read()))))

175615763

In [176]:
# Part 2
sum = 0
skip_next = 0
for match in re.finditer('do\(\)|don\'t\(\)|mul\((\d{1,3}),(\d{1,3})\)', txt):
    match match[0]:
        case "don't()":
            skip_next = 1
            continue
        case "do()":
            skip_next = 0
            continue
        case _:
            if skip_next:
                continue
            else:
                sum += int(match[1]) * int(match[2])
sum

74361272

# DAY 4

In [137]:
matrice = [[w for w in r.strip('\n')] for r in open('files/advent4.txt', 'r').readlines()]

In [138]:
# Part 1
def check(matrice, i, j, depth, direction_i, direction_j):
    if i < 0 or i >= len(matrice) or j < 0 or j >= len(matrice[0]):
        return 0
    if matrice[i][j] == letters[depth]:
        if depth == 3:
            return 1
        else:
            return check(matrice, i + direction_i, j + direction_j, depth + 1, direction_i, direction_j)
    else:
        return 0

letters = ['X', 'M', 'A', 'S']
depth = 0

tot = 0
for i in range(len(matrice)):
    for j in range(len(matrice[i])):
        if matrice[i][j] == letters[depth]:
            for direction_i in [-1, 0, 1]:
                for direction_j in [-1, 0, 1]:
                    tot += check(matrice, i + direction_i, j + direction_j, depth + 1, direction_i, direction_j)
tot

2521

In [139]:
# Part 2
def check(matrice, i, j, checked_other, direction_i, direction_j):
    if i < 0 or i >= len(matrice) or j < 0 or j >= len(matrice[0]):
        return 0
    letter = matrice[i][j]
    if letter not in ['M', 'S']:
        return 0
    if letter == 'M':
        if not checked_other:
            checked_other = 'M'
        else:
            return 1 if checked_other == 'S' else 0
    if letter == 'S':
        if not checked_other:
            checked_other = 'S'
        else:
            return 1 if checked_other == 'M' else 0
    return check(matrice, i - direction_i - direction_i, j - direction_j - direction_j, checked_other, direction_i, direction_j)

tot = 0
for i in range(len(matrice)):
    for j in range(len(matrice[i])):
        if matrice[i][j] == 'A':
            tot += check(matrice, i - 1, j - 1, None, direction_i=-1, direction_j=-1) and check(matrice, i -1 , j + 1, None, direction_i=-1, direction_j=1)

tot

1912

# DAY 5

In [27]:
rules, updates = open('files/advent5.txt').read().split('\n\n')
updates = [x.split(',') for x in updates.split('\n')]

from functools import cmp_to_key

# -1 x<y, 0 x==y, 1 x > y
compare_by_rules = cmp_to_key(lambda x, y: 1 - 2 * (f'{x}|{y}' in rules))

sum_part1 = sum([int(u[len(u) // 2]) for u in updates if u == sorted(u, key=compare_by_rules)])
sum_part2 = sum([int(u[len(u) // 2]) for u in updates if u != sorted(u, key=compare_by_rules)])
   
sum_part1, sum_part2

(7307, 4672)

# DAY 6

In [195]:
file = 'files/advent6.txt'
map = [[i for i in j if i != '\n'] for j in open(file).readlines()]
obstacles = [[i, j] for i, _ in enumerate(open(file).readlines()) for j, t in enumerate(_) if t =='#']

In [232]:
# Part 1

from collections import defaultdict
symb, (x, y) = [(t, [i, j]) for i, _ in enumerate(open(file).readlines()) for j, t in enumerate(_) if t not in ['#', '.', '\n']][0]
def get_direction(symb):
    if symb == '^':
        direction_i, direction_j = -1, 0
    elif symb == '>':
        direction_i, direction_j = 0, 1
    elif symb == 'v':
        direction_i, direction_j = 1, 0
    elif symb == '<':
        direction_i, direction_j = 0, -1
    return direction_i, direction_j

evolution_symb = {'^': '>', '>': 'v', 'v':'<', '<': '^'}
seen = defaultdict(list) | {(x,y): [(-1, 0)]}
def run_sim(x, y, symb, seen, new_obstacle):
    obstacles_f = obstacles + [new_obstacle]
    direction_i, direction_j = get_direction(symb)
    while x >= 0 and x < len(map) and y >= 0 and y < len(map[0]):
        while ([x + direction_i, y + direction_j] not in obstacles_f) and (x >= 0 and x < len(map) and y >= 0 and y < len(map[0])):
            x += direction_i
            y += direction_j
            if (x, y) in seen:
                if (direction_i, direction_j) in seen[(x,y)]:
                    return [new_obstacle]
            seen[(x, y)].append((direction_i, direction_j))
        symb = evolution_symb[symb]
        direction_i, direction_j = get_direction(symb)
    return []

run_sim(x, y, symb, seen, [])
len(seen) - 1

4656

In [233]:
# Part 2
from copy import deepcopy
wrong_paths = []
symb, (x, y) = [(t, [i, j]) for i, _ in enumerate(open(file).readlines()) for j, t in enumerate(_) if t not in ['#', '.', '\n']][0]
seen = defaultdict(list) | {(x,y): [(-1, 0)]}
direction_i, direction_j = get_direction(symb)

while x >= 0 and x < len(map) and y >= 0 and y < len(map[0]):
    while ([x + direction_i, y + direction_j] not in obstacles) and (x >= 0 and x < len(map) and y >= 0 and y < len(map[0])):
        if [x + direction_i, y + direction_j] not in wrong_paths and (x + direction_i, y + direction_j) not in seen:
            seen_cp = deepcopy(seen)
            wrong_paths += run_sim(x, y, symb, seen_cp, [x + direction_i, y + direction_j])
        x += direction_i
        y += direction_j

        seen[(x, y)].append((direction_i, direction_j))
    symb = evolution_symb[symb]
    direction_i, direction_j = get_direction(symb)
len(wrong_paths)

1575

# DAY 7

In [33]:
from itertools import product
from functools import reduce

parsed_input = [(
    int(l.split(': ')[0]),
    list(map(int, l.split(': ')[1].split(' ')))) for l in open('files/advent7.txt').readlines()]

ope = {'+': lambda x, y: x + y, '*': lambda x, y : x * y, '||': lambda x,y: int(str(x) + str(y))}

count = 0
for result, digits in parsed_input:
    first_digit = digits.pop(0)
    for combinations in product(['+', '*'], repeat=len(digits)):
        if result == reduce(lambda acc, iv: ope[combinations[iv[0]]](acc, iv[1]), enumerate(digits), first_digit):
            count += result
            break
count

1153997401072

In [32]:
# Part 2 v1
parsed_input = [(
    int(l.split(': ')[0]),
    list(map(int, l.split(': ')[1].split(' ')))) for l in open('files/advent7.txt').readlines()]

count = 0
for result, digits in parsed_input:
    first_digit = digits.pop(0)
    for combinations in product(['+', '*', '||'], repeat=len(digits)):
        if result == reduce(lambda acc, iv: ope[combinations[iv[0]]](acc, iv[1]), enumerate(digits), first_digit):
            count += result
            break
count

97902809384118

In [41]:
# Part 2 v2
parsed_input = [(
    int(l.split(': ')[0]),
    list(map(int, l.split(': ')[1].split(' ')))) for l in open('files/advent7.txt').readlines()]

count = 0

for result, digits in parsed_input:
    l_res = [digits.pop(0)]
    for d in digits:
        l_res = [f(old_res, d) for old_res in l_res for f in ope.values()]
    if result in l_res:
        count += result
count

97902809384118

# DAY 8

In [90]:
from collections import defaultdict
from operator import add, sub

map = [[i for i in j if i != '\n'] for j in open('files/advent8.txt')]
antennas = defaultdict(list)
for i, row in enumerate(map):
    for j, val in enumerate(row):
        if val != '.':
            antennas[val].append((i, j))     

In [91]:
# Part 1

anti_nodes = []
for antenna, l_coord in antennas.items():
    for comb in combinations(l_coord, 2):
        dist_x, dist_y = comb[0][0] - comb[1][0],  comb[0][1] - comb[1][1]
        for i, op in enumerate([add, sub]):
            n = (op(comb[i][0], dist_x), op(comb[i][1], dist_y))
            if n[0] >= 0 and n[0] < len(map) and n[1] >= 0 and n[1] <len(map[0]) and n not in anti_nodes:
                anti_nodes.append(n)
len(anti_nodes)

369

In [92]:
# Part 2

anti_nodes = []
for antenna, l_coord in antennas.items():
    for comb in combinations(l_coord, 2):
        dist_x, dist_y = comb[0][0] - comb[1][0],  comb[0][1] - comb[1][1]
        for i, op in enumerate([add, sub]):
            if comb[i] not in anti_nodes:
                anti_nodes.append(comb[i])
            n = (op(comb[i][0], dist_x), op(comb[i][1], dist_y))
            while n[0] >= 0 and n[0] < len(map) and n[1] >= 0 and n[1] <len(map[0]):
                if n not in anti_nodes:
                    anti_nodes.append(n)
                n = (op(n[0], dist_x), op(n[1], dist_y))

len(anti_nodes)

1169

# DAY 9

In [56]:
# Part 1
l = open('files/advent9.txt').read()
#l = '2333133121414131402'

free_indexes = list(map(int, list(l)[1::2]))
files = list(map(int, list(l)[::2]))

filled_memory = []
for i, memory_size in enumerate(free_indexes):
    if len(filled_memory) == len(files) - 1:
        break
    filled_memory_idx = []
    while memory_size != 0:
        last_to_fit = files.pop()
        if memory_size >= last_to_fit:
            filled_memory_idx.extend([len(files)] * last_to_fit)
            memory_size = memory_size - last_to_fit
        else:
            filled_memory_idx.extend([len(files)] * memory_size)
            files.append(last_to_fit - memory_size)
            memory_size = 0
    filled_memory.append(filled_memory_idx)

final = []
for i, nb_file in enumerate(files):
    final.extend([i] * nb_file)
    if i < len(filled_memory):
        final.extend(filled_memory[i])
        
sum([i * nb for i, nb in enumerate(final)])


6395800119709

In [85]:
# Part 2
from tqdm import tqdm

l = open('files/advent9.txt').read()

free_indexes = list(map(int, list(l)[1::2]))
files = list(map(int, list(l)[::2]))

from collections import defaultdict

file_for_given_memory = defaultdict(list)
for memory_size in set(free_indexes):
    for i, file_size in enumerate(files):
        if file_size <= memory_size:
            file_for_given_memory[memory_size].append(i)


sorted = []
memory_size_gone = []
file_checked_for_memory = defaultdict(list)
filled_memory = []
for i, memory_size in tqdm(list(enumerate(free_indexes))):
    filled_memory_idx = []
    for i_f, file in enumerate(files[:i:-1]):
        if memory_size == 0 or memory_size in memory_size_gone:
            break
        if len(files) - i_f - 1 in sorted or len(files) - i_f - 1 not in file_for_given_memory[memory_size]:
            continue
        filled_memory_idx.extend([len(files) - i_f - 1] * file)
        memory_size = memory_size - file
        sorted.append(len(files) - i_f - 1)
    if memory_size:
        memory_size_gone.append(memory_size)
        filled_memory_idx.extend(['.'] * memory_size)

    filled_memory.append(filled_memory_idx)

final = []
for i, nb_file in enumerate(files):
    if i not in sorted:
        final.extend([i] * nb_file)
    else:
        final.extend(['.'] * nb_file)
    if i < len(filled_memory):
        final.extend(filled_memory[i])
        
sum([i * nb for i, nb in enumerate(final) if nb != '.'])

100%|██████████| 9999/9999 [02:02<00:00, 81.44it/s] 


6418529470362

In [4]:
from tqdm import tqdm

# Part 2 V2
l = open('files/advent9.txt').read()

free_indexes = list(map(int, list(l)[1::2]))
files = list(map(int, list(l)[::2]))

count, index_sorted = 0, 0
sorted, memory_size_to_not_check = [], []
for i, memory_size in tqdm(list(enumerate(free_indexes))):
    if i not in sorted:
        count += sum([(j + index_sorted) * i for j in range(files[i])])
    index_sorted += files[i]
    for i_f, file_size in enumerate(files[:i:-1]):
        if memory_size == 0 or memory_size in memory_size_to_not_check:
            break
        if len(files) - i_f - 1 in sorted:
            continue
        if file_size <= memory_size:
            count += sum([(index_sorted + j) * (len(files) - i_f - 1) for j in range(file_size)])
            index_sorted += file_size
            memory_size = memory_size - file_size
            sorted.append(len(files) - i_f - 1)
    if memory_size:
        memory_size_to_not_check.append(memory_size)
        index_sorted += memory_size
count

100%|██████████| 9999/9999 [01:49<00:00, 91.24it/s]  


6418529470362

# Day 10

In [92]:
# Part 1

from collections import defaultdict
map = {(i,j): int(v) for i, line in enumerate(open('files/advent10.txt')) for j, v in enumerate(line) if v!='\n' and v!='.'}

grad_pos = defaultdict(list)
for (i,j), v in map.items():
    grad_pos[v].append((i,j))


start_coord_validated = defaultdict(list)

def check_paths(paths, height, start_coord):
    global valid_trailpaths
    if height == 9:
        for (x,y) in paths:
            if (x,y) not in start_coord_validated[start_coord]:
                valid_trailpaths += 1
                start_coord_validated[start_coord].append((x, y))
        return
    for x,y in paths:
        paths_available = set([(x + i, y + j) for (i, j) in [(-1, 0), (0, 1), (1, 0), (0, -1)]]) & set(grad_pos[height + 1])
        check_paths(paths_available, height + 1, start_coord)

valid_trailpaths = 0
for x,y in grad_pos[0]:
    check_paths([(x, y)], 0, (x, y))

valid_trailpaths

514

In [102]:
# Part 2
def check_paths2(paths, height):
    global valid_trailpaths
    if height == 9:
        for (x,y) in paths:
            valid_trailpaths += 1
        return
    for x,y in paths:
        paths_available = set([(x + i, y + j) for (i, j) in [(-1, 0), (0, 1), (1, 0), (0, -1)]]) & set(grad_pos[height + 1])
        check_paths2(paths_available, height + 1)

valid_trailpaths = 0
check_paths2(grad_pos[0], 0)

valid_trailpaths

1162

# DAY 11

In [33]:
from functools import cache

@cache
def blink(d, i=75):
    if i == 0:
        return 1
    if d == 0:
        return blink(1, i - 1)
    elif len(str(d)) % 2 == 0:
        return blink(int(str(d)[:int(len( str(d)) / 2)]), i - 1) + blink(int(str(d)[int(len( str(d)) / 2):]), i - 1)
    else:
        return blink(d * 2024, i - 1)


sum(map(blink, list(map(int,  "70949 6183 4 3825336 613971 0 15 182".split(' ')))))

221280540398419

# DAY 12

In [135]:
# Part 1
from collections import defaultdict

map = {(i,j): v for i, line in enumerate(open('files/advent12.txt')) for j, v in enumerate(line) if v!='\n'}
areas = defaultdict(list)

areas_splitted = defaultdict(list)

in_area = []
for (i,j), v in map.items():
    areas[v].append((i, j))

for v, l_xy in areas.items():
    for (x, y) in l_xy:
        if not areas_splitted[v]:
            areas_splitted[v].append([(x, y)])
            in_area.append((x, y))
            continue
        else:
            matching_area = []
            for i, started_area in enumerate(areas_splitted[v]):
                if set([(x + i, y + j) for (i, j) in [(-1, 0), (0, 1), (1, 0), (0, -1)]]) & set(started_area):
                    matching_area.append(i)
                    
            if len(matching_area) == 1:
                areas_splitted[v][matching_area[0]].append((x, y))
                in_area.append((x, y))
                
            elif len(matching_area) == 2:
                areas_splitted[v][matching_area[0]] = areas_splitted[v][matching_area[0]] + areas_splitted[v].pop(matching_area[1]) + [(x, y)]
                in_area.append((x, y))
                
            if (x, y) not in in_area:
                areas_splitted[v].append([(x, y)])
                in_area.append((x, y))
    
def get_perim(area):
    return sum([(x + i, y + j) not in area for x,y in area for i,j in [(-1, 0), (0, 1), (1, 0), (0, -1)]])

sum([get_perim(area) * len(area) for k, area_l in areas_splitted.items() for area in area_l])

1371306

In [136]:
# Part 2
def count_side(l_x_y, direction_x, direction_y):
    tot = 0
    counted = []
    for (x,y) in l_x_y:
        if (x,y) not in counted:
            tot += 1
        while (x + direction_x, y + direction_y) in l_x_y:
            
            counted.append((x + direction_x, y + direction_y))
            x, y = x + direction_x, y + direction_y
    return tot


def get_side(area):
    if len(area) in [1, 2]:
        return 4
    else:
        sides_c = defaultdict(list)
        for (x, y) in area:
            for dir, i, j in [('up', -1, 0), ('down', 1, 0), ('right', 0, 1), ('left', 0, -1)]:
                if (x + i, y + j) not in area:
                    sides_c[dir].append((x, y))
                    
        return sum([count_side(sides_c['left'], 1, 0), count_side(sides_c['right'], 1, 0), 
                    count_side(sides_c['up'], 0, 1),  count_side(sides_c['down'], 0, 1)])
  

    
sum([get_side(area) * len(area) for area_l in areas_splitted.values() for area in area_l])

805880

# DAY 13

In [131]:
# PART 1
import re

tot = 0
for claw in open('files/advent13_test.txt').read().split('\n\n'):
    a, b, prize = [tuple(map(int, re.findall('\d+', a))) for a in claw.split('\n')]

    if a[0] * 100 + b[0] * 100 < prize[0] or a[1] * 100 + b[1] * 100 < prize[1]:
        continue
    else:
        winning_comb = []
        for i in range(100):
            for j in range(100):
                if prize == (a[0] * i + b[0] * j, a[1] * i + b[1] * j):
                    winning_comb.append((i, j))
        if not winning_comb:
            continue
        tot += min([3 * w[0] + w[1] for w in winning_comb])

tot

480

In [134]:
# PART 2, works for PART 1 too
import sympy as sym

tot = 0
for claw in open('files/advent13.txt').read().split('\n\n'):
    a, b, prize = [tuple(map(int, re.findall('\d+', a))) for a in claw.split('\n')]
    prize = (prize[0] + 10000000000000, prize[1] + 10000000000000)
    x,y = sym.symbols('x,y', integer=True)
    res = sym.solve([x * a[0] + y * b[0] - prize[0], x * a[1] + y * b[1] - prize[1]], (x, y))
    tot += 3 * res[x] + res[y] if res else 0
    
tot

79352015273424

# DAY 14

In [3]:
# PART 1
from collections import defaultdict
from functools import reduce
import regex as re

max_x, max_y = 101, 103
counts = defaultdict(int)
for l in open('files/advent14.txt'):
    (x,y), (vx, vy)= (tuple(map(int, re.findall('-?\d+', a))) for a in l.split(' '))
    (x_100, y_100) = (x + 100 * vx) % max_x, (y + 100 * vy) % max_y
    counts[(x_100 < (max_x - 1) / 2, y_100 < (max_y - 1) / 2)] += 1 if not (x_100 == (max_x - 1) / 2 or y_100 == (max_y - 1) / 2) else 0

reduce(lambda cum, x: cum * x, counts.values(), 1)

231019008

In [193]:
# PART 2 - result oriented assuming tree is being displayed when nodes are not left alone
from tqdm import tqdm

max_x, max_y = 101, 103
def get_counts(i):
    values = {}
    for l in open('files/advent14.txt'):
        (x,y), (vx, vy) = (tuple(map(int, re.findall('-?\d+', a))) for a in l.split(' '))
        (x_i, y_i) = (x + i * vx) % max_x, (y + i * vy) % max_y
        values[(x_i, y_i)] = 1
    return values

def count_neighbours(values):
    is_alone = 0
    for x,y in values.keys():
        is_alone += sum([(x + i, y + j) not in values for i in [-1, 0, 1] for j in (-1, 0, 1) if (i, j) != (0, 0)]) == 8
    return is_alone

i = 0
neighbours = []
for i in tqdm(range(max_x * max_y)):
    values = get_counts(i)
    neighbours.append((count_neighbours(values), i))
    
min(neighbours)


100%|██████████| 10403/10403 [00:35<00:00, 293.97it/s]


(127, 8280)

# DAY 15

In [26]:
map, directions = open('files/advent15.txt').read().split('\n\n')
directions = directions.replace('\n', '')
map =  [[v for v in line] for line in map.split('\n')]
coords_symb = {(i, j): v for i, line in enumerate(map) for j, v in enumerate(line)}
map_directions = {'<': (0, -1), '^': (-1, 0), ">": (0, 1), 'v': (1, 0)}


def move(x, y, symb):
    dir_x, dir_y = map_directions[symb]
    if coords_symb[(x + dir_x, y + dir_y)] == '.':
        coords_symb[(x, y)] = '.'
        coords_symb[(x + dir_x, y + dir_y)] = '@'
        return (x + dir_x, y + dir_y)
    elif coords_symb[(x + dir_x, y + dir_y)] == '#':
        return (x, y)
    elif coords_symb[(x + dir_x, y + dir_y)] == 'O':
        start_x, start_y = (x, y)
        while coords_symb[(x + dir_x, y + dir_y)] == 'O':
            x, y = x + dir_x, y + dir_y
        if coords_symb[(x + dir_x, y + dir_y)] == '#':
            return (start_x, start_y)
        else:
            coords_symb[(x + dir_x, y + dir_y)] = 'O'
            coords_symb[(start_x, start_y)] = '.'
            coords_symb[(start_x + dir_x, start_y + dir_y)] = '@'
            return (start_x + dir_x, start_y + dir_y)
            
(x, y) = [(i, j) for (i, j), k in coords_symb.items() if k == '@'][0]
for i, symb in enumerate(directions):
    x, y = move(x, y, symb)
sum([100 * i + j for (i, j), s in coords_symb.items() if s == 'O'])


1349898

In [122]:
#PART 2 Nasty job here

# debugging func
def print_c(coords):
    for i in range(len(l_map)):
        l = ''
        for j in range(len(l_map[0])):
            l = l + coords[(i, j)]
        print(l)

map, directions = open('files/advent15.txt').read().split('\n\n')
directions = directions.replace('\n', '')
l_map = []
for line in map.split('\n'):
    m = []
    for v in line:
        if v in ['#', '.']:
            m.append(v)
            m.append(v)
        elif v == 'O':
            m.append('[')
            m.append(']')
        else:
            m.append(v)
            m.append('.')
    l_map.append(m)

coords_symb = {(i, j): v for i, line in enumerate(l_map) for j, v in enumerate(line)}
map_directions = {'<': (0, -1), '^': (-1, 0), ">": (0, 1), 'v': (1, 0)}



def is_pushable(x, y, dir_x, dir_y, visited=[]):
    visited.append((x, y))
    if coords_symb[(x + dir_x, y + dir_y)] == "#":
        return 1
    elif coords_symb[(x + dir_x, y + dir_y)] == '[':
        return is_pushable(x + dir_x, y + dir_y, dir_x, dir_y, visited) + is_pushable(x + dir_x, y + dir_y + 1, dir_x, dir_y, visited)
    elif coords_symb[(x + dir_x, y + dir_y)] == ']':
        return is_pushable(x + dir_x, y + dir_y, dir_x, dir_y, visited) + is_pushable(x + dir_x, y + dir_y - 1, dir_x, dir_y, visited)
    else:
        return 0
    
            
def move(x, y, symb):
    dir_x, dir_y = map_directions[symb]
    if coords_symb[(x + dir_x, y + dir_y)] == '.':
        coords_symb[(x, y)] = '.'
        coords_symb[(x + dir_x, y + dir_y)] = '@'
        return (x + dir_x, y + dir_y)
    elif coords_symb[(x + dir_x, y + dir_y)] == '#':
        return (x, y)
    elif coords_symb[(x + dir_x, y + dir_y)] in ('[', ']'):
        start_x, start_y = (x, y)
        if dir_x == 0:
            # usual things from part 1 with new symbols
            while coords_symb[(x + dir_x, y + dir_y)] in ('[', ']'):
                x, y = x + dir_x, y + dir_y
            if coords_symb[(x + dir_x, y + dir_y)] == '#':
                return (start_x, start_y)
            else:
                coords_symb[(start_x, start_y)] = '.'
                while (x, y) != (start_x, start_y):
                    coords_symb[(x + dir_x, y + dir_y)] = coords_symb[(x, y)]
                    (x, y) = x -dir_x, y - dir_y
                coords_symb[(start_x + dir_x, start_y + dir_y)] = '@'
                return (start_x + dir_x, start_y + dir_y)
        else:
            # need to check recursively if movable then move all intricated boxes
            visited = []
            nb_blocked = is_pushable(x, y, dir_x, dir_y, visited)
            if nb_blocked != 0:
                return (x, y)
            else:
                to_clean = (x + dir_x, y + dir_y + 1) if coords_symb[(x + dir_x, y + dir_y)] == '[' else (x + dir_x, y + dir_y - 1)
                to_update = [(x_v + dir_x, y_v + dir_y) for (x_v, y_v) in visited]
                done = []
                for (i, j) in visited[::-1]:
                    if (i, j) != (x, y) and (i, j) not in done:
                        coords_symb[(i + dir_x, j + dir_y)] = coords_symb[(i, j)]
                        done.append((i, j))
                        if (i, j) not in to_update:
                            coords_symb[(i, j)] = '.'
                coords_symb[(x, y)] = '.'
                coords_symb[(x + dir_x, y + dir_y)] = '@'
                coords_symb[to_clean] = '.'
                return (x + dir_x, y + dir_y)
            
(x, y) = [(i, j) for (i, j), k in coords_symb.items() if k == '@'][0]
for i, symb in enumerate(directions):
    # print(i, symb)
    # print_c(coords_symb)
    x, y = move(x, y, symb)
sum([100 * i + j for (i, j), s in coords_symb.items() if s == '['])
#print_c(coords_symb)

1376686

In [6]:

nums1 = [1,2,3,0,0,0]
nums2 = [2,5,6]
sorted(nums1 + nums2)


[0, 0, 0, 1, 2, 2, 3, 5, 6]

# DAY 16

In [263]:
# PART 1 + 2
from collections import defaultdict
from heapq import heapify, heappop, heappush
map =  [[v for v in line] for line in open('files/advent16.txt').read().split('\n')]
coords_symb = {(i, j): v for i, line in enumerate(map) for j, v in enumerate(line) if v != "#"}


(x, y) = [(i, j) for (i, j), k in coords_symb.items() if k == 'S'][0]
(end_x, end_y) = [(i, j) for (i, j), k in coords_symb.items() if k == 'E'][0]

shortest_distance = defaultdict(lambda: float('inf'))


def find_shortest(start_x, start_y):
    min_d = float('inf')
    distances = [(0, (start_x, start_y), (0, 1), [(start_x, start_y)])]
    paths = []
    heapify(distances)
    while distances:
        d, (x,y), (dir_x, dir_y), p = heappop(distances)
        if d > min_d:
            break
        if (x, y, dir_x, dir_y) in shortest_distance and d > shortest_distance[(x, y, dir_x, dir_y)]:
            continue
        if coords_symb[(x, y)] == 'E':
            min_d = d
            paths.extend(p)
        shortest_distance[(x, y, dir_x, dir_y)] = d
        if (x + dir_x, y + dir_y) in coords_symb:
            heappush(distances, (d + 1, (x + dir_x, y + dir_y), (dir_x, dir_y), p + [(x + dir_x, y + dir_y)]))
        if (x + dir_y, y + dir_x) in coords_symb:
            heappush(distances, (d + 1000, (x, y), (dir_y, dir_x), p + [(x, y)]))
        if (x - dir_y, y - dir_x) in coords_symb:
            heappush(distances, (d + 1000, (x, y), (-dir_y, -dir_x), p + [(x, y)]))

    return min_d, paths

shortest, paths = find_shortest(x, y)
shortest, len(set(paths))

(105496, 524)

# DAY 17

In [32]:
import re

registers, program = open('files/advent17.txt').read().split('\n\n')
a, b, c = map(int, re.findall('\d+', registers))
program = list(map(int, program.split(': ')[1].split(',')))

        
def execute(a, b, c):
    output = []
    i = 0
    while i < len(program):
        operand = program[i + 1]
        combo = [0,1,2,3,a,b,c,99999][operand]
        match program[i]:
            case 0:
                a = int(a / 2**combo)
            case 1:
                b = b ^ operand
            case 2:
                b = combo % 8
            case 3:
                i = operand - 2 if a else i
            case 4:
                b = b ^ c
            case 5:
                output.append(combo % 8)
            case 6:
                b = int(a / 2**combo)
            case 7:
                c = int(a / 2**combo)
        i += 2
    return output
print(','.join(map(str, execute(a, b, c))))


# PART 2 - from 4HbQ
valid_a = []
def find(a, i):
    if execute(a, b, c) == program: valid_a.append(a)
    if execute(a, b, c) == program[-i:] or not i:
        for n in range(8): 
            find(8*a+n, i+1)
        
find(0, 0)
min(valid_a)

5,1,3,4,3,7,2,1,7


216584205979245

# DAY 18

In [134]:
# PART 1
from collections import deque
import re

map_size = 70
fallen_bytes = 1024

falling_bytes = [(tuple(map(int, re.findall('\d+', a)))) for a in open('files/advent18.txt').read().split('\n')]
start_x, start_y = (0, 0)
end_x, end_y = (map_size, map_size)


def visit(fallen_bytes, x=start_x, y=start_y):
    coords_symb = {(x, y) for x in range(map_size + 1) for y in range(map_size + 1) if (y, x) not in falling_bytes[:fallen_bytes]}
    seen = set()
    q = deque([(0, (start_x, start_y))])
    min_d = 100_000
    while q:
        d, (x, y) = q.popleft()
        if (x, y) == (end_x, end_y):
            min_d = d
            break
        for dir_x, dir_y in [(0,1), (0, -1), (1, 0), (-1, 0)]:
            if (x + dir_x, y + dir_y) in coords_symb and (x + dir_x, y + dir_y) not in seen:
                seen.add((x + dir_x, y + dir_y))
                q.append((d + 1, (x + dir_x, y + dir_y)))
    return min_d

visit(1024)

280

In [137]:
#PART 2
from bisect import bisect

falling_bytes[bisect(range(len(falling_bytes)), 99_999, key=visit) - 1]

(28, 56)

# DAY 19

In [130]:
from functools import cache

towels, designs = open('files/advent19.txt').read().split('\n\n')
towels, designs = towels.split(', '), designs.split('\n')
max_towel_size = max([len(t) for t in towels])

@cache
def get_combinations(word, depth=0):
    if depth == 0:
        return 1
    tot = 0
    for i in range(1, max_towel_size + 1)[::-1]:
        first_part = word[:i]
        if first_part in towels:
            tot += get_combinations(word[i:], depth - i)
    return tot

sum(get_combinations(d, len(d)) > 0 for d in designs), sum(get_combinations(d, len(d)) for d in designs)

(336, 758890600222015)

# DAY 20

In [516]:
from collections import defaultdict

path_coord = {(x, y): v for x, line in enumerate(open('files/advent20.txt').read().split('\n')) for y, v in enumerate(line) if v != '#'}
wall_coord = [(x, y) for x, line in enumerate(open('files/advent20.txt').read().split('\n')) for y, v in enumerate(line) if v == '#']
cheats = defaultdict(list)

x, y = [(x, y) for (x, y), k in path_coord.items() if k == 'S'][0]
last_x, last_y = None, None
end_x, end_y = [(x, y) for (x, y), k in path_coord.items() if k == 'E'][0]

dist = 1
distances = {(x, y): 0}
while (x, y) != (end_x, end_y):
    for (dir_x, dir_y) in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        if (x + dir_x, y + dir_y) in path_coord and (x + dir_x, y + dir_y) != (last_x, last_y):
            distances[(x + dir_x, y + dir_y)] = dist
            dist += 1
            (last_x, last_y) = x, y
            (x, y) = x + dir_x, y + dir_y
            break

for (x, y) in wall_coord:
    paths = []
    for (dir_x, dir_y) in [(-1, 0), (1, 0), (0, -1), (0, 1)]:
        if (x + dir_x, y + dir_y) in distances:
            paths.append((x + dir_x, y + dir_y))

    if len(paths) > 1:
        max_cheat = max(abs(distances[(x0, y0)] - distances[(x1, y1)]) - (abs(x0 - x1) + abs(y0 - y1)) for ((x0, y0), (x1, y1)) in combinations(paths, 2))
        cheats[max_cheat].append((x, y))
        
sum(len(l_coords) for k, l_coords in cheats.items() if k >= 100)

1441

In [517]:
# PART 2 - can change a few structure to make code leanier but easier to debug this way
cheats = defaultdict(list)
seen = set()

def get_points_within_distance(max_distance):
    return [(x, y) for x in range(-max_distance, max_distance +1) for y in range(-(max_distance - abs(x)), max_distance - abs(x)  + 1)]

l_dir_xy = get_points_within_distance(20)

for (x, y) in path_coord:
    paths = []  
    for (dir_x, dir_y) in l_dir_xy:
        if (x + dir_x, y + dir_y) in distances:
            paths.append((x + dir_x, y + dir_y))
    if len(paths) > 1:
        for i, (e_x, e_y) in enumerate(paths):
            if ((e_x, e_y), (x, y)) in seen:
                continue
            dist = abs(distances[(x, y)] - distances[(e_x, e_y)]) - (abs(x - e_x) + abs(y - e_y))
            cheats[dist].append(((x, y), (e_x, e_y)))
            seen.add(((x, y), (e_x, e_y)))

sum(len(l_coords) for k, l_coords in cheats.items() if k >= 100)

1021490

# DAY 21

In [27]:
import re
from functools import cache

# inputs = ["029A", "980A", "179A", "456A", "379A"]
inputs = open('files/advent21.txt').read().split('\n')

num_pad = {'7': (0, 0), '8': (0, 1), '9': (0, 2),
           '4': (1, 0), '5': (1, 1), '6': (1, 2),
           '1': (2, 0), '2': (2, 1), '3': (2, 2),
           'X': (3, 0), '0': (3, 1), 'A': (3, 2)}

dir_pad = {'X': (0, 0), '^': (0, 1), 'A': (0, 2),
           '<': (1, 0), 'v': (1, 1), '>': (1, 2)}


@cache
def get_path_length(current_symb, end_symb, depth, part1=True):
    max_depth = 2 if part1 else 25
    dict_path = num_pad if depth == 0 else dir_pad
    pos, pos_end, pos_blocked = dict_path[current_symb], dict_path[end_symb], dict_path['X']
    dist_x, dist_y = pos[0] - pos_end[0], pos[1] - pos_end[1]
    move_x, move_y = '^' if dist_x >= 0 else 'v', '<' if dist_y >= 0 else '>'
    prioritize_y = (move_y == '<' and not (dist_y and pos[1] - dist_y == pos_blocked[1] and pos[0] == pos_blocked[0])
                            or
                    move_x in '^v' and dist_x and pos[0] - dist_x == pos_blocked[0] and pos[1] == pos_blocked[1])
    s_path =  move_y * abs(dist_y) + move_x * abs(dist_x) + 'A' if prioritize_y else move_x * abs(dist_x) + move_y * abs(dist_y) + 'A'
    
    if depth == max_depth:
        return len(s_path)
    else:
        length, pos_symb = 0, 'A' 
        for symb in s_path:
            length += get_path_length(pos_symb, symb, depth + 1, part1)
            pos_symb = symb
        return length

tot = 0
for input in inputs:
    l, pos = 0, 'A'
    for c in input:
        l += get_path_length(pos, c, 0, part1=False)
        pos = c
    tot += l * int(''.join(list(re.findall('\d', input))))

    
tot

231309103124520

# DAY 22

In [123]:
from functools import cache
from collections import defaultdict
sequences = defaultdict(int)

@cache
def compute_secret(secret, i):
    mix = lambda a,b: a ^ b
    prune = lambda a: a % 16777216
    val = lambda a: int(str(a)[-1])
    if i == 0:
        return secret
    else:
        last_secret = secret
        secret = prune(mix(secret * 64, secret))
        secret = prune(mix(int(secret / 32), secret))
        secret = prune(mix(secret * 2048, secret))
        s_i.append((val(secret), val(secret) - val(last_secret)))
        return compute_secret(secret, i - 1)
    
tot = 0
for s in open('files/advent22.txt').read().split('\n'):
    s_i = []
    seen_i = set()
    # Recursion error with 2000
    tot += compute_secret(compute_secret(int(s), 1000), 1000)
    for i in range(len(s_i)):
        seq = s_i[i: i + 4]
        key = tuple([a[1] for a in seq])
        if len(seq) == 4:
            bananas = seq[-1][0]
            if key not in seen_i: 
                sequences[key] += bananas
                seen_i.add(key)
                
tot, max(sequences.values())
 

(14392541715, 1628)

# DAY 23

In [127]:
from collections import defaultdict
from itertools import combinations
relations = defaultdict(list)
for l in open('files/advent23.txt').read().split('\n'):
    first, second = l.split('-')
    relations[first].append(second)
    relations[second].append(first)

def check(trio):
    for i in range(len(trio)):
        if not all([trio[i] in relations[other] for other in trio[:i] + trio[i + 1:]]):
            return False
    return True


In [128]:
# PART 1
trios = set()
for first_r, l_r in relations.items():
    if first_r.startswith('t'):
        for duo in combinations(l_r, 2):
            if check((first_r,) + duo):
                trios.add(tuple(sorted((first_r,) + duo)))
len(trios)

1304

In [129]:
# PART 1 - one liner
len(set(tuple(sorted((first_r,) + duo)) for first_r, l_r in relations.items() for duo in combinations(l_r, 2) if first_r.startswith('t') and check((first_r, ) + duo)))


1304

In [31]:
# PART 2
sorted_relations = dict(sorted(relations.items(), key=lambda item: len(item[1])))
max_relation = ()
for first_r, l_r in sorted_relations.items():
    max_relation_r = ()
    for comb_size in range(len(l_r))[::-1]:
        if comb_size + 1 <= len(max_relation_r):
            continue
        for others in combinations(l_r, comb_size):
            if check((first_r,) + others):
                if len(others) + 1 > len(max_relation_r):
                    max_relation_r = tuple(sorted((first_r,) + others))
    if len(max_relation_r) > len(max_relation):
        max_relation = max_relation_r
        
','.join(max_relation)

'ao,es,fe,if,in,io,ky,qq,rd,rn,rv,vc,vl'

# DAY 24

In [94]:
init_values, instructions = open('files/advent24.txt').read().split('\n\n')

val = {line.split(':')[0]: int(line.split(':')[1]) for line in init_values.split('\n')}

def clean_instruction(instruction):
    gate, res = instruction.split(' -> ')
    return tuple(gate.split(' ') + [res])

instructions_q = list(map(clean_instruction, instructions.split('\n')))

wrong_instructions = set()
max_z = sorted(instructions_q, key=lambda a: a[-1], reverse=True)[0][-1]

# lcsddit reddit account rules - verify if instructions checks with normal ripple carry adder behaviour
for w0, ope, w1, res in instructions_q:
    if res[0] == 'z' and ope != 'XOR' and res != max_z:
        wrong_instructions.add(res)
    if (
        ope == "XOR"
        and res[0] not in ["x", "y", "z"]
        and w0[0] not in ["x", "y", "z"]
        and w1[0] not in ["x", "y", "z"]
    ):  
        wrong_instructions.add(res)
    if ope == "AND" and "x00" not in [w0, w1]:
        for subw0, subope, subw1, _ in instructions_q:
            if (res == subw0 or res == subw1) and subope != "OR":
                wrong_instructions.add(res)
    if ope == "XOR":
         for subw0, subope, subw1, _ in instructions_q:
            if (res == subw0 or res == subw1) and subope == "OR":
                wrong_instructions.add(res)
                
#PART 1
while instructions_q:
    w0, ope, w1, res = instructions_q.pop(0)
    gate = f'{w0} {ope} {w1}'.replace('AND', '&').replace('XOR', '!=').replace('OR', '|').replace(w0, f"val['{w0}']").replace(w1, f"val['{w1}']")
    if w0 in val and w1 in val:
        val[res] = int(eval(gate))
    else:
        instructions_q.append((w0, w1, ope, res))

s = sum( val[z] * 2 ** i for i, z in enumerate(sorted([w for w in val if w.startswith('z')])))
s, ','.join(sorted(wrong_instructions))

(58367545758258, 'bpf,fdw,hcc,hqc,qcw,z05,z11,z35')

# DAY 25

In [125]:
locks, keys = [], []
for block_raw in open('files/advent25.txt').read().split('\n\n'):
    block = block_raw.split('\n')
    is_lock = True if all(b == '#' for b in block[0]) else False
    if not is_lock:
        block = block[::-1]
    s = [sum(block[j][i] == '#' for j in range(len(block))) for i in range(len(block[0]))]
    locks.append(s) if is_lock else keys.append(s)
    
sum(not any(k + l > len(block) for k, l in zip(key, lock)) for key in keys for lock in locks)

3287